# Information Retrieval - Pandemic Investigation  
This notebook retrieves abstracts relevant to pandemics and then uses topic modeling to analyze the chosen abstracts.  Three info retrieval techniques are used: Literal Term Matching, TF-IDF, and Latent Semantic Indexing.  These are linear algebra techniques.  
We use the Scikit-Learn library.

In [ ]:
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt
import gensim
import time

from sklearn.decomposition import NMF, TruncatedSVD, LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import pairwise_distances

#from gensim.models.coherencemodel import CoherenceModel

import TextCleaning

In [ ]:
# pull in raw abstracts

raw_df=pd.read_csv('../../data/original/raw_abstracts.csv',engine='python')

# remove null abstracts and duplicates

df = TextCleaning.remove_nulls(raw_df, "ABSTRACT")
df = TextCleaning.remove_duplicates(df)

df.reset_index(inplace = True)
df.rename(columns={'index':'original index'}, inplace=True)

In [ ]:
df.head()

In [ ]:
# input needed for doc-term matrix creation in Scikit-Learn is one string per document (not a list of strings).  
# Original data is already in this form!

docs = df["ABSTRACT"] 

## Functions needed for all info retrieval approaches

In [ ]:
# Create query vector 

def create_query(words, terms):
    
    # words: search query words
    # terms: terms in corpus
    
    q = np.zeros(len(terms))  # number of terms

    idx = []
    for word in query_words:
        idx.append(terms.index(word))

    q[idx] = 1
    
    return q

In [ ]:
def return_top_abstracts(docs, scores, top_n):
    
    '''
    docs: Series that contains abstract
    scores: scores of abstracts
    top_n: return the top_n abstracts given by idx
    '''
    # sort scores in descending order
    scores_sorted_idx = np.argsort(scores)[::-1]
    
    ix = scores_sorted_idx[:top_n]
    print(ix[0:10])
    
    return ix, docs[ix]
    

In [ ]:
def create_result_df(abstracts, scores):
    
    df = pd.DataFrame()
    df["abstracts"] = abstracts
    df["scores"] = scores
    
    return df

## Literal Term Matching - Frequency Count Document-Term Matrix

This will return all abstracts in the corpus with exact word matches to the query.  

Results will be return in sorted order of how high the query scores with each abstract. A high score means more occurences of the query words in the abstract.

In [ ]:
# Create document-term matrix based on count frequencies

vectorizer = CountVectorizer()
doc_term_matrix = vectorizer.fit_transform(docs)

In [ ]:
terms = vectorizer.get_feature_names()

### Create Query Words - list the search terms

A query is just a list of words to search for in the corpus.  We will use the same query for all three info retrieval techniques.

In [ ]:
# CHANGE QUERY WORDS HERE

query_words = ["pandemic"] # other example: ['pandemic', influenza', 'mers', 'sars', 'zikv', 'denv', 'hiv', 'aids']

q = create_query(query_words, terms)

In [ ]:
# calculate the score for each document against the query. Docs with more occurences of the query words 
# will score higher

f_scores = doc_term_matrix.dot(q)

In [ ]:
sum(f_scores >0)  # how many abstracts include at least one of the query words

In [ ]:
# sort scores in descending order

f_scores_sorted = np.sort(f_scores)[::-1]
f_scores_sorted[:10]

In [ ]:
f_idx, f_top_abstracts = return_top_abstracts(docs, f_scores, 200)  # CHANGE NUMBER OF TOP DOCS RETURNED

In [ ]:
f_top_abstracts

In [ ]:
f_top_abstracts.iloc[0]

In [ ]:
f_df = create_result_df(docs, f_scores)

In [ ]:
f_df

## TF-IDF Document-Term Matrix

This approach is similar to Literal Term Matching using frequency counts in the document-term matrix.  However, instead of using frequency counts, the entries of the document-term matrix are weighted using TF-IDF.

In [ ]:
# Find doc-term matrix using TF-IDF weighting

tf_idf_vectorizer = TfidfVectorizer()
tf_idf = tf_idf_vectorizer.fit_transform(docs)

In [ ]:
tf_idf_terms = tf_idf_vectorizer.get_feature_names()  # these terms are the same as the terms created from the 
                                                      # frequency count document-term matrix, so we do not need to
                                                      # recreate the query vector.

In [ ]:
# calculate the score for each document against the query. Docs with more occurences of the query words 
# will score higher

tf_idf_scores = tf_idf.dot(q)

In [ ]:
sum(tf_idf_scores >0)   # how many abstracts include at least one of the query words

In [ ]:
# sort scores in descending order

tf_idf_scores_sorted = np.sort(tf_idf_scores)[::-1]
tf_idf_scores_sorted[:10]

In [ ]:
tfidf_idx, tfidf_top_abstracts = return_top_abstracts(docs, tf_idf_scores, 200)  # CHANGE NUMBER OF TOP DOCS RETURNED

In [ ]:
tfidf_top_abstracts

In [ ]:
tf_idf_df = create_result_df(docs, tf_idf_scores)

In [ ]:
tf_idf_df

## Latent Semantic Indexing (LSI) Approach

LSI Uses the TF-IDF matrix.  LSI is a tecnique that utilizes a truncated Singular Value Decomposition of the document-term matrix.  Basically, LSI still returns relevant documents to the query; however some of the documents returned may not include the exact search terms!  LSI is finding the latent or hidden relationships in the terms.  

In [ ]:
# Find the Truncated SVD of the TF-IDF matrix

lsa = TruncatedSVD(n_components=500, random_state=1)  # CHANGE THE NUMBER OF COMPONENTS - NOTE: MORE COMPONENTS 
                                                      # GIVES YOU A MORE ACCURATE APPROXIMATION OF THE DOC-TERM 
                                                      # MATRIX, BUT IS ALSO MORE EXPENSIVE AND MAY NOT LEAD TO THE 
                                                      # BEST INFO RETRIEVAL RESULTS.
USigma = lsa.fit_transform(tf_idf)
Vtrans = lsa.components_

In [ ]:
# transform query to be in same space as documents

q = q.reshape(1,-1)
qhat = lsa.transform(q)

In [ ]:
print(qhat.shape)
print(USigma.shape)
print(Vtrans.shape)

In [ ]:
lsa_scores = pairwise_distances(qhat, USigma, metric='cosine', n_jobs=7)  # CHANGE N_JOBS TO BE NUMBER OF CORES - 1

In [ ]:
lsa_scores.shape

In [ ]:
lsa_scores

In [ ]:
type(lsa_scores)

In [ ]:
sum(lsa_scores[0] > 0)  # how many abstracts scored above 0

In [ ]:
lsa_scores[0]

In [ ]:
# sort scores in descending order

lsa_scores_sorted = np.sort(lsa_scores[0])[::-1]
lsa_scores_sorted[:10]

In [ ]:
lsa_idx, lsa_top_abstracts = return_top_abstracts(docs, lsa_scores[0], 200)  # CHANGE NUMBER OF TOP DOCS RETURNED

In [ ]:
lsa_top_abstracts

In [ ]:
lsa_df = create_result_df(docs, lsa_scores[0])

In [ ]:
lsa_df

## Create pandemics corpus

We use the results of our three information retrieval techniques to create a new, smaller corpus that only contains abstracts relevant to the query. 

In [ ]:
docs_ix = np.concatenate([f_idx, tfidf_idx, lsa_idx])

In [ ]:
docs_idx = np.unique(docs_ix)

In [ ]:
docs_idx.shape

In [ ]:
lim_docs = [docs[i] for i in docs_idx]

In [ ]:
# print out the abstracts of the pandemics corpus -- long text output!

#lim_docs

## Topic Modeling with relevant pandemic abstracts

In [ ]:
# Create a TF-IDF document-term matrix for the pandemics corpus 

# TRY DIFFERENT PARAMETERS IN THE TF-IDF DOC-TERM MATRIX SET-UP
nmf_vectorizer = TfidfVectorizer(stop_words='english', #ngram_range=(1,2) , 
                                 max_df=0.4, min_df=3, lowercase=True) #, max_features=int(len(lim_docs)/2))

nmf_tf_idf = nmf_vectorizer.fit_transform(lim_docs)

In [ ]:
# topic modeling with NMF

nmf_model = NMF(n_components=10, random_state=1)  # TRY DIFFERENT NUMBERS OF TOPICS
W = nmf_model.fit_transform(nmf_tf_idf)
H = nmf_model.components_

In [ ]:
# TRY TOPIC MODELING WITH LDA



In [ ]:
# function slightly modified from https://nlpforhackers.io/topic-modeling/

def print_topics(model, vectorizer, top_n=10):
    for idx, topic in enumerate(model.components_):  # loop through each row of H.  idx = row index.  topic = actual row
        print("\nTopic %d:" % (idx))
        #print([(vectorizer.get_feature_names()[i], topic[i])  # printing out words corresponding to indices found in next line
                        #for i in topic.argsort()[:-top_n - 1:-1]])  # finding indices of top words in topic
            
        print_list = [(vectorizer.get_feature_names()[i], topic[i])  
                        for i in topic.argsort()[:-top_n - 1:-1]]
        for item in print_list:
            print(item)
        

In [ ]:
print_topics(nmf_model, nmf_vectorizer, 10)